# Empirical analyses

* Loads in and joins processed+modeled brain data.

* Performs boostrapping.

* Creates and saves dataframes that support the figures.

In [1]:
import os
import os.path as op
from glob import glob
import nibabel as nib
from scipy import io
import numpy as np
from numpy.random import SeedSequence, default_rng
import pandas as pd
import itertools
import utils
import time
import ipyparallel as ipp

_______________
## Set up

Parallelization

In [2]:
cluster = ipp.Cluster(n=4)
cluster.start_cluster_sync()

Using existing profile dir: '/Users/sfavila/.ipython/profile_default'
Starting 4 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


In [3]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(4); rc.ids

100%|█████████████████████████████████████████| 4/4 [00:06<00:00,  1.72s/engine]


[0, 1, 2, 3]

In [4]:
dv = rc[:]
v = rc.load_balanced_view()

Assign project paths and variables

In [5]:
data_dir = op.join('..', 'data')
df_dir = op.join(data_dir, 'dataframes')

In [6]:
subjects, rois, tasks = utils.default_data()

Assign random seed

In [7]:
entropy = 157245829812966997872450835235695796168 
ss = SeedSequence(entropy)

------------------------
## Data Loading 

### Behavior
Load subject .csv files into dataframes

#### Load stim assignment

In [8]:
n_stim = 4
stim_info = []
for s_i, subj in enumerate(subjects):
    stim_file = op.join(data_dir,  subj, 'behav', 'stim_info.csv')
    s = pd.read_csv(stim_file).sort_values(by='stim_id').set_index('stim_id')
    stim_info.append(s)

#### Load behavior

*Learning*

In [9]:
learn_behav = []
for s_i, subj in enumerate(subjects):
    behav_files = glob(op.join(data_dir, subj, 'behav', 'learn', 'data_block??.csv'))
    for b in behav_files:
        df = pd.read_csv(b)
        df['subj'] = subj
        learn_behav.append(df)
learn_behav = pd.concat(learn_behav, sort=True)

*Scan*

In [10]:
scan_behav = []
for s_i, subj in enumerate(subjects):
    behav_files = glob(op.join(data_dir, subj, 'behav', 'scan', 'data_block??.csv'))
    for b in behav_files:
        df = pd.read_csv(b)
        df['subj'] = subj
        scan_behav.append(df)
scan_behav = pd.concat(scan_behav, sort=True)

Write out compiled behavioral data

In [11]:
if not op.exists(df_dir):
    os.makedirs(df_dir)
learn_behav.to_csv(op.join(df_dir, 'behav_learn_data.csv'), index=False)
scan_behav.to_csv(op.join(df_dir, 'behav_scan_data.csv'), index=False)

### Brain

Load .mgz and .mat file data into dicts

#### Load retinotopy parameters 

Load lh and rh retinotopy parameters from .mgz files and combine into bilateral

In [12]:
ret_model = 'css'
ret_params = ['full-xcrds', 'full-ycrds', 'full-eccen', 'full-angle', 
              'full-sigma', 'full-vexpl', 'full-prfbeta']
ret = {p:[] for p in ret_params}

for param, subj in itertools.product(ret_params, subjects):
    rdir = op.join(data_dir, subj, 'retinotopy', ret_model)
    rfile_lh = op.join(rdir, 'lh.{}.mgz').format(param)
    r_lh = nib.load(rfile_lh).get_fdata().squeeze()
    
    rfile_rh = op.join(rdir, 'rh.{}.mgz').format(param)
    r_rh = nib.load(rfile_rh).get_fdata().squeeze()
    
    ret[param].append(np.concatenate([r_lh, r_rh]))

#### Load GLMdenoise results

Load glmdenoise params. The .mat file, which contains bilateral (concatenated) parameters is used here, but .mgz  files for the beta weights are also provided with dataset

In [13]:
glm_df = {t: {'beta':[], 'se':[], 'R2':[], 'conds':[]} for t in tasks}

for task, subj in itertools.product(tasks, subjects):
    
    # Load required from mat files
    gfile = op.join(data_dir, subj, 'glmdenoise', task, 'results.mat') 
    g = io.loadmat(gfile)
    gparams = {'beta':       g['modelmd'][0][1], 
               'se':         g['modelse'][0][1],
               'R2':         g['R2'],
               'conds':     [g[0][0] for g in g['conds']]}
    
    # Save conditions/design matrix columns
    glm_df[task]['conds'].append(gparams['conds'])
    
    # For voxelwise parameters save bilateral data
    for param in ['beta', 'se', 'R2']:
        glm_df[task][param].append(gparams[param])

#### Load prf predictions

Load lh and rh prf predictions from .mgz files and combine lh and rh into bilateral

In [14]:
pred_ret_models = ['onegaussian', 'css', 'dog']
pred_keys = ['pred_'+m for m in pred_ret_models]
for m in pred_keys:
    glm_df[m] = []

for mod, subj in itertools.product(pred_ret_models, subjects):
    pdir = op.join(data_dir, subj, 'retinotopy', mod)
    pred_files_lh = glob(op.join(pdir, 'lh.full-pred??.mgz'))
    pred_lh = np.vstack([nib.load(p).get_fdata().squeeze() for p in pred_files_lh]).T
    
    pred_files_rh = glob(op.join(pdir, 'rh.full-pred??.mgz'))
    pred_rh = np.vstack([nib.load(p).get_fdata().squeeze() for p in pred_files_rh]).T
    
    glm_df['pred_'+mod].append(np.concatenate([pred_lh, pred_rh]))

#### Load ROI masks

Load lh and rh rois masks from .mgz files and combine lh and rh into bilateral

In [15]:
masks = {roi:[] for roi in rois}
for roi, subj in itertools.product(rois, subjects):
    mdir = op.join(data_dir, subj, 'rois')
    mfile_lh = op.join(mdir, 'lh.{}_hand.mgz').format(roi)
    m_lh = nib.load(mfile_lh).get_fdata().squeeze().astype(bool)
    
    mfile_rh = op.join(mdir, 'rh.{}_hand.mgz').format(roi)
    m_rh = nib.load(mfile_rh).get_fdata().squeeze().astype(bool)
    
    masks[roi].append(np.concatenate([m_lh, m_rh]))

Restrict all ROIs by eccentricity and  variance explained

In [16]:
eccen_min = .5
eccen_max = 8
vexpl_thresh = .1

for roi, subj in itertools.product(rois, subjects):
    s_i = subjects.index(subj)
    mask = masks[roi][s_i]
    
    eccen = ret['full-eccen'][s_i]
    vexpl = ret['full-vexpl'][s_i]

    mask[vexpl<=vexpl_thresh] = False
    exclude_eccen = np.logical_or(eccen<=eccen_min, eccen>eccen_max)
    mask[exclude_eccen] = False
    
    masks[roi][s_i] = mask

_______________
## Organize experiment and retinotopy data frames

#### Experiment data

Mask experimental brain data to just ROIs of interest and join with behavior into dataframes

In [17]:
exp_df = []
for subj, roi, task in itertools.product(subjects, rois, tasks):
    
    s_i = subjects.index(subj)
    
    # Get roi mask
    mask = masks[roi][s_i]
    
    # Get regressors from glm and corresponding stim info
    conds = glm_df[task]['conds'][s_i]
    sinfo = stim_info[s_i].loc[conds]
    
    # Get masked betas, ses, and prf model predictions
    beta = glm_df[task]['beta'][s_i][mask, :]
    se = glm_df[task]['se'][s_i][mask, :]
    pred = [glm_df[m][s_i][mask, :] for m in pred_keys]
    
    # Split betas by condition
    cond_beta = np.hsplit(beta, beta.shape[1])
    cond_se = np.hsplit(se, se.shape[1])
    cond_pred = [np.hsplit(p, p.shape[1]) for p in pred]
    cond_all = zip(cond_beta, cond_se, *cond_pred)
    
    # Make dataframe
    df = [pd.DataFrame({'subj': subj, 'task': task, 'hemi': 'b', 'roi': roi, 
                        'beta': c[0].squeeze(), 
                        'se': c[1].squeeze(), 
                         pred_keys[0]: c[2].squeeze(),
                         pred_keys[1]: c[3].squeeze(), 
                         pred_keys[2]: c[4].squeeze(), 
                        'vert': np.where(mask.flatten()==True)[0], 
                        'stim_id': sinfo.index[c_i],
                        'stim_angle_brain': sinfo['stim_angle_brain'][c_i],
                        'stim_eccen': sinfo['stim_eccen'][c_i],
                        'stim_xpos': sinfo['stim_xpos'][c_i],
                        'stim_ypos': sinfo['stim_ypos'][c_i]})
         for c_i, c in enumerate(cond_all)]

    exp_df.append(pd.concat(df))
exp_df = pd.concat(exp_df).reset_index(drop=True)

In [18]:
exp_df = exp_df[['subj', 'hemi', 'roi', 'vert', 'task', 'stim_eccen', 'stim_id', 
                 'stim_angle_brain', 'stim_xpos', 'stim_ypos', 'beta', 'se'] + pred_keys]

#### Retinotopy data

Mask retinotopy data to just ROIs of interest

In [19]:
ret_df = []
for subj, roi, param in itertools.product(subjects, rois, ret_params):
    
    s_i = subjects.index(subj)
    
    # Get ret param and mask to roi
    r = ret[param][s_i]
    mask = masks[roi][s_i]
    ret_data = r[mask]
    
    # Make dataframe
    df = pd.DataFrame(dict(subj=subj, task=task, hemi='b', roi=roi,
                           ret_model=ret_model, value=ret_data, param=param, 
                           vert=np.where(mask.flatten()==True)[0]))
    ret_df.append(df)
ret_df = pd.concat(ret_df).reset_index(drop=True)

# Make params wide
ind_cols = ['subj', 'ret_model', 'hemi', 'roi', 'vert']
ret_df = ret_df.pivot_table(values=['value'], index=ind_cols, columns='param').reset_index()
ret_df.columns = ind_cols + ret_df.columns.get_level_values(1)[5:].tolist()

Merge experiment and retinotopy data on cortical surface vertex

In [20]:
vert_data = exp_df.join(ret_df.set_index(['subj', 'hemi', 'roi', 'vert']), on=['subj', 'hemi', 'roi', 'vert'])

---------------------------
## Generate spatial response profiles

### Group spatial response profiles for perception and memory data

Calculate angular distance between each pRF center and each stimulus and bin distances

In [21]:
vert_data = utils.calc_ang_distance(vert_data, exclude_eccen=True)

Calculate eccentricity bins (supplementary analysis)

In [22]:
vert_data = utils.calc_eccen_bin(vert_data, exclude_angle=True)

Write out complete dataframe with all subject ret and experimental data as well as angular distances

In [23]:
vert_data.to_csv(op.join(df_dir, 'vertex_data.csv.gz'), index=False)

Normalize angular distance data, then fit a difference of two von Mises distributions to each roi and task, and write this info out.

In [24]:
#For fitting, drop vertices excluded from both angular distance and eccen bin groups
vert_data = vert_data.dropna(how='all', subset=['ang_dist_bin', 'eccen_bin']) 

norm_data = utils.norm_group(vert_data)
norm_data.to_csv(op.join(df_dir, 'group_ang_data.csv'), index=False)

params = utils.fit_diff_vonmises(norm_data, 'beta_adj')
params.to_csv(op.join(df_dir, 'group_ang_fits.csv'), index=False)

Normalize eccen bin data (supplementary analysis with no fits) and write this info out.

In [25]:
norm_data_eccen = utils.norm_group(vert_data, xvar='eccen_bin')
norm_data_eccen.to_csv(op.join(df_dir, 'group_eccen_data.csv'), index=False)

### Bootstrapping

Create bootstrapped datasets by resampling subjects with replacement (parallelized) and generate spatial response profiles for each boot.

In [26]:
#Push only the data columns we need to reduce memory usage
core_cols = ['subj', 'hemi', 'roi', 'vert', 'task', 'stim_id', 
             'stim_angle_brain', 'full-angle', 'ang_dist_bin', 
             'eccen_bin', 'beta', 'se']
vd = vert_data[core_cols+pred_keys]   #include prf predictions so we can resample predictions and data at once

dv.push({'vert_data':vd, 'subjects':subjects})
time.sleep(1)

In [27]:
def bootstrap_data(stream):  
    
    import pandas
    
    n, rng = stream
    
    # Sample subject ids with replacement
    boot_subj = rng.choice(subjects, len(subjects), replace=True)
    
    # Create new df with these subjects
    boot_data = []
    for i, s in enumerate(boot_subj):
        df = vert_data.query("subj==@s")
        df = df.assign(subj=i+1, orig_subj=s, n_boot=n) 
        boot_data.append(df)
    boot_data = pandas.concat(boot_data).reset_index(drop=True)
    
    return boot_data

Create 500 bootstrapped datasets across engines

In [28]:
n_boots = 500
child_seeds = ss.spawn(n_boots)
streams = [default_rng(s) for s in child_seeds]

boot_data = v.map(bootstrap_data, enumerate(streams), ordered=False)

Normalize data and fit von mises (angular distance data only) to each boot

In [29]:
boot_norm, boot_eccen_norm, boot_params = [], [], []
for b in boot_data:
    
    # angular distance
    n = utils.norm_group(b, group_cols=['n_boot'])
    p = utils.fit_diff_vonmises(n, 'beta_adj', group_cols=['n_boot'])
    boot_norm.append(n)
    boot_params.append(p)
    
    # eccen bins
    n_e = utils.norm_group(b, xvar='eccen_bin', group_cols=['n_boot'])
    boot_eccen_norm.append(n_e)

Save angular distance group data and params for each boot

In [30]:
boot_norm = pd.concat(boot_norm).reset_index(drop=True)
boot_norm = boot_norm.sort_values(by=['n_boot', 'roi', 'task'])
boot_norm.to_csv(op.join(df_dir, 'group_ang_data_boots.csv'), index=False)

boot_params = pd.concat(boot_params).reset_index(drop=True)
boot_params = boot_params.sort_values(by=['n_boot', 'roi', 'task'])
boot_params.to_csv(op.join(df_dir, 'group_ang_fits_boots.csv'), index=False)

Save eccen group data for each boot

In [31]:
boot_eccen_norm = pd.concat(boot_eccen_norm).reset_index(drop=True)
boot_eccen_norm = boot_eccen_norm.sort_values(by=['n_boot', 'roi', 'task'])

boot_eccen_norm.to_csv(op.join(df_dir, 'group_eccen_data_boots.csv'), index=False)

In [32]:
rc.purge_everything()

### Individual subject spatial response profiles

Create average baseline-corrected response profiles for each subject and roi

In [33]:
subj_df = []
for (subj, hemi, roi, task), g in vd.groupby(['subj', 'hemi', 'roi', 'task']):
    avg = g.groupby(['subj', 'hemi', 'roi', 'task', 'ang_dist_bin']).median().reset_index()
    avg['beta_adj'] = avg['beta'] - avg.query("ang_dist_bin>=160 | ang_dist_bin<=-160")['beta'].mean() 
    subj_df.append(avg)
subj_df = pd.concat(subj_df).reset_index(drop=True)

In [34]:
subj_df = subj_df[['subj', 'hemi', 'roi', 'task', 'ang_dist_bin', 'beta_adj']]
subj_df.to_csv(op.join(df_dir, 'subj_ang_data.csv'), index=False) 

In [35]:
subj_fits = utils.fit_diff_vonmises(subj_df, 'beta_adj', xvar='ang_dist_bin', group_cols=['subj'])

In [36]:
subj_fits.to_csv(op.join(df_dir, 'subj_ang_fits.csv'), index=False) 

###  pRF prediction spatial response profiles

Normalize predicted data for each prf model, fit von Mises, and write this info out

In [37]:
pred_norm, pred_params = [], []
for pred_mod in pred_keys:
    
    n = utils.norm_group(vert_data.query("task=='perception'"), yvar=pred_mod)
    pred_norm.append(n)
    
    p = utils.fit_diff_vonmises(n, pred_mod+'_adj', drop_cols='task')
    p.loc[:, 'prf_model'] = pred_mod
    pred_params.append(p)

In [38]:
pred_norm = pd.concat(pred_norm, sort=True)
pred_norm = pred_norm.melt(id_vars=['hemi', 'roi', 'ang_dist_bin', 'ang_dist_bin_rad'], 
                           value_vars=['pred_onegaussian_adj', 'pred_css_adj', 'pred_dog_adj'],
                           var_name='prf_model', value_name='beta_adj').dropna().reset_index(drop=True)
pred_norm['prf_model'] = pred_norm['prf_model'].apply(lambda x:x[:-4])
pred_norm.to_csv(op.join(df_dir, 'pRFpred_group_ang_data.csv'), index=False) 


pred_params = pd.concat(pred_params).reset_index(drop=True)
pred_params.to_csv(op.join(df_dir, 'pRFpred_group_ang_fits.csv'), index=False)

Normalize and fit von Mises to predictions in each bootstrap (boots computed in the bootstrapping section above)

In [39]:
pred_boot_norm, pred_boot_params = [], []
for pred_mod, b in itertools.product(pred_keys, boot_data):
    
    n = utils.norm_group(b.query("task=='perception'"), yvar=pred_mod, group_cols=['n_boot'])
    pred_boot_norm.append(n)
    
    p = utils.fit_diff_vonmises(n, pred_mod+'_adj', group_cols=['n_boot'])
    p.loc[:, 'prf_model'] = pred_mod
    pred_boot_params.append(p)

/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in multiply
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in double_scalars
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: invalid value encountered in true_divide
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in exp
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in multiply
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0

/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in double_scalars
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in multiply
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: invalid value encountered in true_divide
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in exp
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0,kappa))
/Volumes/server/Projects/Spatial_Memory/public/osf/analysis_notebooks/utils.py:71: RuntimeWarning: overflow encountered in multiply
  p = scale * np.exp(kappa*np.cos(theta-loc))/(2*np.pi*iv(0

In [40]:
pred_boot_norm = pd.concat(pred_boot_norm, sort=True)
pred_boot_norm = pred_boot_norm.melt(id_vars=['n_boot', 'hemi', 'roi', 'ang_dist_bin'], 
                                     value_vars=['pred_onegaussian_adj', 'pred_css_adj', 'pred_dog_adj'],
                                     var_name='prf_model', value_name='beta_adj').dropna().reset_index(drop=True)
pred_boot_norm['prf_model'] = pred_boot_norm['prf_model'].apply(lambda x:x[:-4])
pred_boot_norm = pred_boot_norm.sort_values(by=['n_boot', 'roi', 'prf_model'])
pred_boot_norm.to_csv(op.join(df_dir, 'pRFpred_group_ang_data_boots.csv'), index=False)


pred_boot_params = pd.concat(pred_boot_params).reset_index(drop=True)
pred_boot_params = pred_boot_params.sort_values(by=['n_boot', 'roi', 'prf_model'])
pred_boot_params.to_csv(op.join(df_dir, 'pRFpred_group_ang_fits_boots.csv'), index=False)